<a href="https://colab.research.google.com/github/hezarai/notebooks/blob/main/hezar/02_train_a_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install hezar

# Training a model in Hezar

In this notebook, we're gonna demonstrate a training walkthrough. Training a model in Hezar is pretty much like any other library or even simpler! As mentioned before, any model in Hezar is also a PyTorch module. So training a model is actually training a PyTorch model with some more cool features! Lets dive in.

In [1]:
from hezar.models import BertTextClassification, BertTextClassificationConfig
from hezar.data import Dataset
from hezar.trainer import Trainer, TrainerConfig
from hezar.preprocessors import Preprocessor

/home/aryan/Applications/miniconda3/envs/main/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATASET_PATH = "hezarai/sentiment-dksf"  # dataset path on the Hub
BASE_MODEL_PATH = "hezarai/bert-base-fa"  # used as model backbone weights and tokenizer

### Build the datasets

Lets load a dataset from Hub. For this example we use the Digikala/SnappFood comments datasets which is used for sentiment analysis.

In [3]:
train_dataset = Dataset.load(DATASET_PATH, split="train", tokenizer_path=BASE_MODEL_PATH)
eval_dataset = Dataset.load(DATASET_PATH, split="test", tokenizer_path=BASE_MODEL_PATH)

### Build the model

Choose a model for this task and build the model as you would normally do in Hezar (See [models overview](01_models_overview.ipynb))

In [4]:
model = BertTextClassification(BertTextClassificationConfig(id2label=train_dataset.config.id2label))

We also need the tokenizer for our model.

In [5]:
tokenizer = Preprocessor.load(BASE_MODEL_PATH)

### Training

Hezar comes with a built-in `Trainer` so that model training is as easy and straightforward as possible. As you might have guessed, in order to use a Trainer we first need to setup the config.

In [6]:
train_config = TrainerConfig(
    output_dir="bert-sentiment-fa",
    task="text_classification",
    device="cuda",
    init_weights_from=BASE_MODEL_PATH,
    batch_size=8,
    num_epochs=5,
    checkpoints_dir="checkpoints/",
    metrics=["accuracy", "f1"],
    num_dataloader_workers=0,
    seed=42,
    optimizer="adam",
    learning_rate=2e-5,
    lr_scheduler="reduce_lr_on_plateau",
    save_freq=1,
)

Notice that our model is a BERT model with random weights, but we want to finetune it for a simple task. So we need to load the pretrained language model weights. To do this, simply provide the `init_weights_from` parameter which takes a Hub ID to a model and loads the weights to our model. (Missing classification head is automatically ignored)

Now that we have our config, lets build the Trainer.

In [7]:
trainer = Trainer(
    config=train_config,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=train_dataset.data_collator,
    preprocessor=tokenizer,
)

Hezar (WARNING): Partially loading the weights as the model architecture and the given state dict are incompatible! 
Ignore this warning in case you plan on fine-tuning this model
Incompatible keys: []
Missing keys: ['classifier.weight', 'classifier.bias']



Aaaannnddd lets train!

In [8]:
trainer.train()


******************** Training Info ********************

  Output Directory: `bert-sentiment-fa`
  Task: `text_classification`
  Model: `BertTextClassification`
  Init Weights: `hezarai/bert-base-fa`
  Device(s): `cuda`
  Training Dataset: `TextClassificationDataset(path=hezarai/sentiment-dksf['train'], size=28602)`
  Evaluation Dataset: `TextClassificationDataset(path=hezarai/sentiment-dksf['test'], size=2315)`
  Optimizer: `adam`
  Initial Learning Rate: `2e-05`
  Learning Rate Decay: `0.0`
  Epochs: `5`
  Batch Size: `8`
  Number of Parameters: `118299651`
  Number of Trainable Parameters: `118299651`
  Mixed Precision: `Full (fp32)`
  Metrics: `['accuracy', 'f1']`
  Checkpoints Path: `bert-sentiment-fa/checkpoints/`
  Logs Path: `bert-sentiment-fa/logs`

*******************************************************




Evaluating...   100%|######################################################################| 289/289 [00:07<00:00, 37.84batch/s, accuracy=0.839, f1=0.744, loss=0.411]


Evaluating...   100%|######################################################################| 289/289 [00:07<00:00, 39.09batch/s, accuracy=0.761, f1=0.657, loss=0.566]


Evaluating...   100%|######################################################################| 289/289 [00:07<00:00, 37.68batch/s, accuracy=0.871, f1=0.806, loss=0.371]


Evaluating...   100%|######################################################################| 289/289 [00:07<00:00, 38.70batch/s, accuracy=0.875, f1=0.809, loss=0.388]


Evaluating...   100%|######################################################################| 289/289 [00:07<00:00, 37.69batch/s, accuracy=0.848, f1=0.77, loss=0.58]  
Hezar (INFO): Training done!


So we trained the model for 5 epochs. As you can see, everything is verbosed during the process. After each epoch all metrics and weights are logged and saved. Tensorboard logs are saved to a folder called `runs` (you can change this default) and you can inspect it as usual:

In [ ]:
%tensorboard --logdir runs/

And the weights are saved to `checkpoints` (you can change this default).

### Push to Hub

Now we can push our model along with some training specific configs to the Hub! 

In [ ]:
trainer.push_to_hub("hezarai/bert-fa-sentiment-digikala-snappfood")